# xbatcher / pytorch dataloader turtorial

If you have not installed xbatcher, can be done via pip or conda. The docs for xbatcher are here:

https://xbatcher.readthedocs.io/en/latest/

In [1]:
# !pip install xbatcher

# or

# conda install -c conda-forge xbatcher

In [2]:
import numpy as np

import xarray as xr

# importing xbatcher
import xbatcher
from xbatcher import BatchGenerator
from xbatcher.loaders.torch import IterableDataset

In [3]:
# pytorch specfic loaders
import torch

from torch.utils.data import TensorDataset, DataLoader

In [4]:
# nice way to show imported packages
from watermark import watermark
%load_ext watermark

In [5]:
%watermark --iversions

xbatcher: 0.3.0
torch   : 1.13.0
xarray  : 2022.11.0
numpy   : 1.23.5



## xarray:

This is the standard toy dataset for xarray testing

In [6]:
ds = xr.tutorial.load_dataset("air_temperature")
print('number of timesteps:', len(ds.time))

# adding a second variable to be somewhat more realistic than a 1 variable dataset
ds['air2'] = ds.air+42.42 #nonsense numbers
ds['target_air'] = ds.air-100 #nonsense numbers

ds

number of timesteps: 2920


<xarray.Dataset>
Dimensions:     (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat         (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon         (lon) float32 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time        (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air         (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2        (time, lat, lon) float32 283.6 284.9 285.9 ... 338.9 338.6 338.1
    target_air  (time, lat, lon) float32 141.2 142.5 143.5 ... 196.5 196.2 195.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## xbatcher intro:

Size of batches of explained [here](https://xbatcher.readthedocs.io/en/latest/demo.html#Controlling-the-size/shape-of-batches) .

### pytorch stuff

PyTorch data loader info: https://pytorch.org/docs/stable/data.html

you can also define batch sizes in pytorch, but will have less control/might not scale as well. We will not be using this feature.

Worth poking around with to_array() to understand how it's dumping out the np arrays

In [7]:
x = ds[['air', 'air2']]
y = ds['target_air']

In [9]:
number_of_batches = 12

x_gen = BatchGenerator(x, {"time": number_of_batches})
y_gen = BatchGenerator(y, {"time": number_of_batches})

In [10]:
dataset = IterableDataset(x_gen, y_gen) # this is an xbatcher method

In [11]:
loader = torch.utils.data.DataLoader(dataset, batch_size=None)

In [12]:
type(loader)

torch.utils.data.dataloader.DataLoader

This dataset should probably be processed, normalized, etc., and maybe a differnt shape would be better for your use case. But this should serve as an example for a relatively simple case to go from a multi-variable xarray dataset to a pytorch data loader. 